# Intro Agents from Scratch

## 3 Levels

1. LLMs + functions in prompt
2. LLMs + structured outputs/function calling
3. Agent loop

### 1. LLMs + functions in prompt

In [1]:
from openai import OpenAI

client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "system", "content": "You are a helpful assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

get_response("Hi! What are the levels of agents?")

'Could you share what domain you mean by “agents”? For example:\n- AI/LLM or robotics agents (autonomy levels)\n- Customer support agents (Tier 1/2/3)\n- Real estate agents (salesperson, broker, managing broker)\n- Game/CS agents (reactive vs deliberative)\n- Level‑k reasoning in multi‑agent games (L0, L1, L2…)\n\nIf you mean AI/LLM agents, a common ladder of “agent levels” is:\n1) Tool-augmented assistant: answers plus single tool calls (search, code exec).\n2) Multi-step planner: decomposes tasks and uses multiple tools/steps.\n3) Reflective agent: iterates, critiques itself, uses memory/context to improve.\n4) Goal-driven autonomous worker: runs long tasks, monitors progress/events, minimal supervision.\n5) Orchestrated multi-agent system: multiple specialized agents coordinated by a controller.\n\nTell me which context you had in mind, and I’ll tailor the levels accordingly.'

In [2]:
def write_file(file_path, content):
    """Takes in a file path and content, and writes the content to the file"""
    with open(file_path, "w") as f:
        f.write(content)

def read_file(file_path):
    """Takes in a file path and returns the content of the file"""
    with open(file_path, "r") as f:
        return f.read()


def level1_agent_llm_and_functions_in_prompt(prompt):
    function_info = """
    def write_file(file_path, content):
        '''Takes in a file path and content, and writes the content to the file'''
        with open(file_path, "w") as f:
            f.write(content)

    def read_file(file_path):
        '''Takes in a file path and returns the content of the file'''
        with open(file_path, "r") as f:
            return f.read()

    """
    full_prompt_with_function_info = f"""
    Take this request from a user: {prompt}.
    If the request involves writing to a file or reading to a file,
    you can output a call to these functions which you have access to: 
    {function_info}.
    """
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "system", "content": "You are a helpful assistant that can write and read files."},
                  {"role": "user", "content": full_prompt_with_function_info}]
    )
    
    return response.choices[0].message.content
    
level1_agent_llm_and_functions_in_prompt("Write a file called 'test.txt' with the content 'Hello, world!'")

'write_file("test.txt", "Hello, world!")'

In [3]:
exec('write_file("test.txt", "Hello, world!")')

## Level 2: LLMs + structured outputs/function calling

In [4]:
from pydantic import BaseModel, Field

class WriteFileOperation(BaseModel):
    file_path: str = Field(description="The path to the file to be written to or read from")
    content: str = Field(description="The content to be written to the file")

class ReadFileOperation(BaseModel):
    file_path: str = Field(description="The path to the file to be written to or read from")


response = client.beta.chat.completions.parse(
    model="gpt-5",
    messages=[{"role": "system", "content": "You are a helpful assistant that can write and read files."},
              {"role": "user", "content": "Write a file called 'test.txt' with the content 'Hello, world!'"}],
    response_format=WriteFileOperation)

output_write_file_ops = response.choices[0].message.parsed

In [5]:
print(output_write_file_ops.file_path)
print(output_write_file_ops.content)

test.txt
Hello, world!


In [6]:
def level2_agent_llm_structured(prompt):
    response = client.beta.chat.completions.parse(
    model="gpt-5",
    messages=[{"role": "system", "content": "You are a helpful assistant that can write and read files."},
              {"role": "user", "content": prompt}],
    response_format=WriteFileOperation)# Structured OUTPUT from the LLM!
    output_args = response.choices[0].message.parsed
    # FUNCTION CALLING!
    write_file(output_args.file_path, output_args.content)
    print("File was created!")
    return output_args

level2_agent_llm_structured("Write a file called 'level2agentoutput.txt' with the content 'Level 2 works!'")

File was created!


WriteFileOperation(file_path='level2agentoutput.txt', content='Level 2 works!')

In [7]:
# save this as sample_input.txt
response = get_response("Create a 3 paragraph exaplanation of modern llm agents")
write_file("sample_input.txt", response)

In [10]:
def level3_agent_loop(task_prompt):
    """
    ReAct-style agent loop that iteratively:
    1. Observes the current state
    2. Thinks about what to do next
    3. Takes an action (either calls a tool or provides final answer)
    4. Gets observation from the action
    5. Repeats until task is complete
    """
    # Available tools that the agent can use
    available_tools = {
        'read_file': read_file,
        'write_file': write_file
    }
    
    # Initialize conversation history with system prompt
    messages = [
        {"role": "system", "content": """You are a helpful assistant that can read and write files.
        
            You work in a loop where you:
            1. Think about what needs to be done
            2. Either call a tool OR provide a final answer
            3. Observe the result and continue if needed

            Available tools:
            - read_file(file_path): Reads content from a file
            - write_file(file_path, content): Writes content to a file

            When you need to use a tool, respond with:
            THOUGHT: [your reasoning]
            ACTION: [tool_name](arguments)

            When you have the final answer, respond with:
            THOUGHT: [your reasoning]
            ANSWER: [your final response to the user]

            Important: Only use ACTION or ANSWER, not both in the same response."""},
        {"role": "user", "content": task_prompt}
    ]
    
    max_iterations = 10  # Prevent infinite loops
    iteration = 0
    
    print(f"User Task: {task_prompt}\n")
    print("-" * 50)
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n--- Iteration {iteration} ---")
        
        # Get LLM response
        response = client.chat.completions.create(
            model="gpt-4o",  # Using a model that exists
            messages=messages,
            temperature=0.1  # Lower temperature for more consistent behavior
        )
        
        agent_response = response.choices[0].message.content
        print(f"Agent: {agent_response}")
        
        # Add agent's response to conversation history
        messages.append({"role": "assistant", "content": agent_response})
        
        # Parse the response to determine if it's an action or final answer
        if "ANSWER:" in agent_response:
            # Agent has provided final answer
            answer = agent_response.split("ANSWER:")[1].strip()
            print(f"\n✓ Final Answer: {answer}")
            return answer
        
        elif "ACTION:" in agent_response:
            # Agent wants to execute a tool
            action_line = agent_response.split("ACTION:")[1].strip().split('\n')[0].strip()
            
            try:
                # Parse the function call
                import re
                match = re.match(r'(\w+)\((.*)\)', action_line)
                if match:
                    tool_name = match.group(1)
                    args_str = match.group(2)
                    
                    if tool_name in available_tools:
                        # Execute the tool
                        print(f"Executing: {action_line}")
                        
                        # Parse arguments (simple parsing for demo)
                        if tool_name == 'read_file':
                            # Extract file path from quotes
                            file_path = args_str.strip('\'"')
                            observation = available_tools[tool_name](file_path)
                        elif tool_name == 'write_file':
                            # Split by comma and extract both arguments
                            parts = args_str.split(',', 1)
                            file_path = parts[0].strip().strip('\'"')
                            content = parts[1].strip().strip('\'"') if len(parts) > 1 else ""
                            available_tools[tool_name](file_path, content)
                            observation = f"Successfully wrote to {file_path}"
                        
                        print(f"Observation: {observation[:200]}..." if len(str(observation)) > 200 else f"Observation: {observation}")
                        
                        # Add observation to conversation
                        messages.append({"role": "user", "content": f"Observation from {tool_name}: {observation}"})
                    else:
                        messages.append({"role": "user", "content": f"Error: Unknown tool '{tool_name}'"})
                else:
                    messages.append({"role": "user", "content": f"Error: Could not parse action '{action_line}'"})
                    
            except Exception as e:
                error_msg = f"Error executing action: {str(e)}"
                print(f"Error: {error_msg}")
                messages.append({"role": "user", "content": error_msg})
        else:
            # Agent provided neither ACTION nor ANSWER - prompt for clarification
            messages.append({"role": "user", "content": "Please respond with either 'ACTION: tool_name(args)' to use a tool or 'ANSWER: your_response' to provide the final answer."})
    
    print("\n⚠ Maximum iterations reached")
    return "Maximum iterations reached without completing the task"
    
# Test the agent loop
prompt = "Read the file: 'sample_input.txt' and write a summary of that file into a new file called 'summary_file.txt'"
result = level3_agent_loop(prompt)

User Task: Read the file: 'sample_input.txt' and write a summary of that file into a new file called 'summary_file.txt'

--------------------------------------------------

--- Iteration 1 ---
Agent: THOUGHT: I need to read the content of 'sample_input.txt' to understand its contents before I can write a summary.
ACTION: read_file('sample_input.txt')
Executing: read_file('sample_input.txt')
Observation: Modern LLM agents are software systems that wrap a large language model in an autonomous loop that can plan, act through tools, and adapt based on feedback to pursue a goal. Unlike static chatbots tha...

--- Iteration 2 ---
Agent: THOUGHT: I have read the content of 'sample_input.txt'. Now, I need to write a concise summary of this content and save it to 'summary_file.txt'.

Summary: The text describes modern LLM agents, which are advanced software systems that utilize large language models to autonomously plan, act, and adapt to achieve goals. Unlike static chatbots, these agents exec